## Cloning the Repository

In [ ]:
!git clone --recursive https://github.com/hoandhh/AIRC_Multimodal_VideoCaptioning.git
%cd AIRC_Multimodal_VideoCaptioning/

Cloning into 'AIRC_Multimodal_VideoCaptioning'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 151 (delta 2), reused 7 (delta 2), pack-reused 138
Receiving objects: 100% (151/151), 239.51 MiB | 33.06 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Updating files: 100% (98/98), done.
/content/AIRC_Multimodal_VideoCaptioning


## Cài đặt thư viện và môi trường

> Thêm khối trích dẫn


Vì trên googlecolab không hỗ trợ conda nên sẽ chạy qua miniconda


Cài đặt miniconda


In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_23.1.0-1-Linux-x86_64.sh -q --show-progress
!bash ./Miniconda3-py37_23.1.0-1-Linux-x86_64.sh -b -f -p /usr/local

Miniconda3-py37_23. 100%[===================>]  86.46M   249MB/s    in 0.3s    
PREFIX=/usr/local
Unpacking payload ...
                                                                               
Installing base environment...





Preparing transaction: - \ | / done
Executing transaction: \ | / - \ | / - \ | / - \ | / - \ | / - \ done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local


Cài đặt các gói bổ sung và cấu hình môi trường:

In [ ]:
#@title
# fixing nasty colab environment
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install cryptography==42.0.5
!pip install pyopenssl --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 44.8 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 38.0.4
    Uninstalling cryptography-38.0.4:
      Successfully uninstalled cryptography-38.0.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: pyopenssl
    Found existing installation: pyOpenSSL 22.0.0
    Uninstalling pyOpenSSL-22.0.0:
      Successfully uninstalled pyOpenSSL-22.0.0


Thêm đường dẫn của Miniconda vào sys.path để Python có thể nhận diện các thư viện cài đặt qua Miniconda:

In [ ]:
import os
from pathlib import Path
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
from sample.single_video_prediction import get_video_duration

Tạo các môi trường conda cho việc trích xuất đặc trưng và mô hình dự đoán:

In [ ]:
# feature extraction
!export PIP_DEFAULT_TIMEOUT=100
!conda env create -f ./submodules/video_features/conda_env_i3d.yml
!conda env create -f ./submodules/video_features/conda_env_vggish.yml
# captioning model
!conda env create -f ./conda_env.yml
# spacy language model
!/usr/local/envs/bmt/bin/python -m spacy download en

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.











 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...




tk-8.6.8             | 3.1 MB    | : 100% 1.0/1 [00:05<00:00,  5.54s/it]               




tk-8.6.8             | 3.1 MB    | : 100% 1.0/1 [00:05<00:00,  5.54s/it]


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...


















 ... (more hidden) ...




## Tải GloVe


In [ ]:
!wget https://a3s.fi/swift/v1/AUTH_a235c0f452d648828f745589cde1219a/bmt/glove.840B.300d.zip -q --show-progress
!wget https://a3s.fi/swift/v1/AUTH_a235c0f452d648828f745589cde1219a/bmt/best_cap_model.pt -q --show-progress
!wget https://a3s.fi/swift/v1/AUTH_a235c0f452d648828f745589cde1219a/bmt/best_prop_model.pt -q --show-progress
!wget https://storage.googleapis.com/audioset/vggish_model.ckpt -q --show-progress

!mkdir .vector_cache
!mv glove.840B.300d.zip ./.vector_cache/
!mv best_cap_model.pt ./sample/
!mv best_prop_model.pt ./sample/
!mv vggish_model.ckpt ./submodules/video_features/models/vggish/checkpoints/

glove.840B.300d.zip 100%[===================>]   2.03G  16.9MB/s    in 2m 20s  
best_cap_model.pt   100%[===================>] 589.61M  15.9MB/s    in 39s     
best_prop_model.pt  100%[===================>]   2.50G  16.8MB/s    in 2m 37s  
vggish_model.ckpt   100%[===================>] 277.62M  70.1MB/s    in 4.0s    


## Chạy thử

Upload video vào file sample

In [ ]:
# upload a video
MY_VIDEO_PATH = '/content/sample_data/test.mp4'

# Preparing the paths
VIDEO_DURATION = get_video_duration(MY_VIDEO_PATH)

FEATURES_CACHE_PATH = '/content/AIRC_Multimodal_VideoCaptioning/tmp/'
FEATURES_PATH_STUB = os.path.join(FEATURES_CACHE_PATH, Path(MY_VIDEO_PATH).stem)
FEATURE_PATH_VGGISH = f'{FEATURES_PATH_STUB}_vggish.npy'
FEATURE_PATH_RGB = f'{FEATURES_PATH_STUB}_rgb.npy'
FEATURE_PATH_FLOW = f'{FEATURES_PATH_STUB}_flow.npy'

PROPOSAL_CKPT = '/content/AIRC_Multimodal_VideoCaptioning/sample/best_prop_model.pt'
CAPTIONING_CKPT = '/content/AIRC_Multimodal_VideoCaptioning/sample/best_cap_model.pt'

Video Duration: 40.890313


The script will extract audio and visual features from the video

In [ ]:
# Extract I3D features (visual)
!cd ./submodules/video_features && /usr/local/envs/i3d/bin/python main.py \
    --feature_type i3d \
    --on_extraction save_numpy \
    --device_ids 0 \
    --extraction_fps 25 \
    --video_paths $MY_VIDEO_PATH \
    --output_path $FEATURES_CACHE_PATH

# Extract VGGish features (audio)
!cd ./submodules/video_features && /usr/local/envs/vggish/bin/python main.py \
    --feature_type vggish \
    --on_extraction save_numpy \
    --device_ids 0 \
    --video_paths $MY_VIDEO_PATH \
    --output_path $FEATURES_CACHE_PATH

Saving features to /content/AIRC_Multimodal_VideoCaptioning/tmp/
100% 1/1 [00:54<00:00, 54.39s/it]
Saving features to /content/AIRC_Multimodal_VideoCaptioning/tmp/

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

100% 1/1 [00:12<00:00, 12.51s/it]


Chạy dense video captioning

In [ ]:
# captioning parameters
MAX_PROP_PER_VIDEO = 100
NMS_TIOU_THRESHOLD = 0.4

# Running single video prediction
!/usr/local/envs/bmt/bin/python ./sample/single_video_prediction.py \
    --prop_generator_model_path $PROPOSAL_CKPT \
    --pretrained_cap_model_path $CAPTIONING_CKPT \
    --vggish_features_path $FEATURE_PATH_VGGISH \
    --rgb_features_path $FEATURE_PATH_RGB \
    --flow_features_path $FEATURE_PATH_FLOW \
    --duration_in_secs $VIDEO_DURATION \
    --device_id 0 \
    --max_prop_per_vid $MAX_PROP_PER_VIDEO \
    --nms_tiou_thresh $NMS_TIOU_THRESHOLD

Contructing caption_iterator for "train" phase
100% 2196016/2196017 [03:53<00:00, 9389.47it/s]
Using vanilla Generator
initialization: xavier
Glove emb of the same size as d_model_caps
Pretrained caption path: 
 /content/AIRC_Multimodal_VideoCaptioning/sample/best_cap_model.pt
[{'start': 0.1, 'end': 4.6, 'sentence': 'A black screen appears with white words on the screen'}, {'start': 38.8, 'end': 40.4, 'sentence': 'A man in a black shirt is standing on a table'}, {'start': 5.3, 'end': 35.9, 'sentence': 'A man is seen speaking to the camera and leads into several shots of people performing various tricks'}, {'start': 29.0, 'end': 38.9, 'sentence': 'We see a man talking'}, {'start': 34.9, 'end': 40.7, 'sentence': 'A man in a black shirt is standing on a stage'}, {'start': 1.4, 'end': 16.5, 'sentence': 'A man is talking in a room and talking to the camera'}, {'start': 0.3, 'end': 2.1, 'sentence': 'We see a title screen'}]
